### 加载数据集

In [ ]:
import sys
sys.path.append('../../')


from datasets.datasets import DatasetManager

dataset_manager = DatasetManager(batch_size=2048)
# train_loader, test_loader = dataset_manager.cifar_dataset()
train_loader, test_loader = dataset_manager.mnist_dataset()

Using device: cuda:7
CIFAR-10 path: /home/jzgrp/huiwang/mio/Code/KAN/datasets/CIFAR10
MNIST path: /home/jzgrp/huiwang/mio/Code/KAN/datasets/mnist


### 加载模型

In [ ]:
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=7)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
# from torchmetrics import Accuracy
import torch.nn.functional as F
from models.models import ModelManager 

model_manager = ModelManager()



class KANKANReLU(nn.Module):
    def __init__(self):
        super(KANKANReLU, self).__init__()
        self.kan1 = model_manager.KANLinear(28*28, 32, base_activation=nn.ReLU)
        self.kan2 = model_manager.KANLinear(32, 10, base_activation=nn.ReLU)
        # self.kan3 = model_manager.KANLinear(32, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 展开图像为一维向量
        x = self.kan1(x)
        x = self.kan2(x)
        # x = self.kan3(x)
        return x
    
class KANKANGELU(nn.Module):
    def __init__(self):
        super(KANKANGELU, self).__init__()
        self.kan1 = model_manager.KANLinear(28*28, 32, base_activation=nn.GELU)
        self.kan2 = model_manager.KANLinear(32, 10, base_activation=nn.GELU)
        # self.kan3 = model_manager.KANLinear(32, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 展开图像为一维向量
        x = self.kan1(x)
        x = self.kan2(x)
        # x = self.kan3(x)
        return x

class KANKANMish(nn.Module):
    def __init__(self):
        super(KANKANMish, self).__init__()
        self.kan1 = model_manager.KANLinear(28*28, 32, base_activation=nn.Mish)
        self.kan2 = model_manager.KANLinear(32, 10, base_activation=nn.Mish)
        # self.kan3 = model_manager.KANLinear(32, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 展开图像为一维向量
        x = self.kan1(x)
        x = self.kan2(x)
        # x = self.kan3(x)
        return x

In [ ]:

from torchinfo import summary
import os


model_test = "KANKANReLU"

model = KANKANReLU().to(device)  # Use 'cuda' for GPU
# model = KANKANGELU().to(device)  # Use 'cuda' for GPU
# model = KANKANMish().to(device)  # Use 'cuda' for GPU
# model = KANKAN().to(device)  # Use 'cuda' for GPU
# model = MLPMLP().to(device)  # Use 'cuda' for GPU


optimizer = optim.LBFGS(model.parameters(), lr=0.01)  # Reduced learning rate from 0.1 to 0.01

# 使用torchsummary输出模型结构
print(summary(model, input_size=(128, 1, 28, 28)))  # 输入特征为128维
# print(summary(model, input_size=(128, 1, 28, 28)))  # 输入特征为128维

if not os.path.exists(f"../../weights/{model_test}/"):
    os.makedirs(f"../../weights/{model_test}/")


### 训练(利用预训练模型可只执行第一步然后跳去评估部分)

In [ ]:
from weights.weights import WeightManager
weight_manager = WeightManager()

In [ ]:
import time

# Define the training loop
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        def closure():
            optimizer.zero_grad()
            output = model(data)
            loss = nn.CrossEntropyLoss()(output, target)
            loss.backward()
            return loss
        data, target = data.to(device), target.to(device)
        optimizer.step(closure)
        if batch_idx % 10 == 0:
            loss = closure()
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

start_time = time.time()
for epoch in range(1, 6):
    # train_lasso(model, device, train_loader, optimizer, epoch, lambda_l1=0.001)
    train(model, device, train_loader, optimizer, epoch)
    weight_manager.save_model(model, optimizer, epoch=epoch, dir_name=model_test, file_name=f'{model_test}_{epoch}_checkpoint.pth')
end_time = time.time()
print(f"Training Time: {end_time - start_time}")


### 评估

#### 读取所有权重文件测试

In [ ]:
# Evaluate the model
def evaluate(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')



pth_files = weight_manager.list_pth_files(dir_name=model_test)

# Evaluate the trained model
if pth_files:
    print("Available .pth files:")
    for pth_file in pth_files:
        print(f"- {pth_file}")
    for pth_file in pth_files:
        model, optimizer, start_epoch = weight_manager.load_model(model, optimizer, dir_name=f'{model_test}', file_name=f"{pth_file}", device=device)
        
        start_time = time.time()
        evaluate(model, device, test_loader)
        end_time = time.time()
        print(f"Inference Time: {end_time - start_time}")

# evaluate(model, device, test_loader)